## [13460](https://www.acmicpc.net/problem/13460)

[유튜브강의](https://www.youtube.com/watch?v=HIJ6ke8RYxg)

메모리 : 32628KB
시간 : 92ms

In [50]:
from collections import deque

class Ball:
    def __init__(self, row: int, col: int, is_red: bool):
        self.r = row
        self.c = col
        self.is_red = is_red
    # d 방향으로 이동한 상태의 Ball 객체
    # 해당하는 방향으로 쭉 이동 -> O를 만나면 거기서 종료, #을 만나면 #이전 상태에서 반환
    def move(self, d: int):
        i = 1
        while True:
            nr = self.r + deltas[d][0]*i
            nc = self.c + deltas[d][1]*i
            # 구멍을 만났다면?
            if map_[nr][nc] == 'O':
                return Ball(nr, nc, self.is_red)
            # 벽을 만나면 벽 이전에서 멈춰야 한다. 그리고 Ball의 위치에 가벽을 세워주자!
            elif map_[nr][nc] == '#':
                new_ball = Ball(nr - deltas[d][0], nc - deltas[d][1], self.is_red)
                map_[new_ball.r][new_ball.c] = '#' # 가벽 세우기 -> 이번 동작이 끝나면 치워주자!
                return new_ball
            i+=1
        
class BallPair:
    def __init__(self, red: Ball, blue: Ball):
        self.red = red
        self.blue = blue
        self.ordered = [red, blue]
    # 북, 남, 서, 동 : 0 ,1 ,2 ,3
    def re_order(self, d: int):
        if d == 0: # 위쪽으로 기울일때, row가 작은녀석이 먼저
            self.ordered[0] = (self.red if self.red.r < self.blue.r else self.blue)
        elif d == 1: # 아래쪽으로 기울일때, row가 큰녀석이 먼저
            self.ordered[0] = (self.red if self.red.r > self.blue.r else self.blue)
        elif d == 2: # 왼쪽으로 기울일때, col이 작은녀석 먼저
            self.ordered[0] = (self.red if self.red.c < self.blue.c else self.blue)
        else: # 오른쪽으로 기울일때, col이 큰녀석 먼저
            self.ordered[0] = (self.red if self.red.c > self.blue.c else self.blue)
        self.ordered[1] = (self.blue if self.ordered[0].is_red else self.red)

def bfs(red: Ball, blue: Ball) -> int:
    queue = deque()
    
    visited = [[[[False] * C for _ in range(R)] for _ in range(C)] for _ in range(R)] # red r, red c, blue r, blue c
    
    # 초기설정
    queue.append(BallPair(red, blue))
    visited[red.r][red.c][blue.r][blue.c] = True
    
    turn = 1
    size = 0
    while queue:
        size = len(queue)
        while size > 0:
            # 가장 앞에 녀석 가져오기
            head = queue.popleft()
            
            # 각 방향으로 이동시켜보기
            for d in range(len(deltas)):
                # 각각의 볼을 이동시킨다 - 누가 먼저?
                head.re_order(d)
                # 선 후공에 따라서 ball들을 이동시키기 - Ball class에 정의
                move_first = head.ordered[0].move(d)
                move_second = head.ordered[1].move(d)
                # 두 공의 이동 과정에서 만들었던 가벽 제거
                if map_[move_first.r][move_first.c] == '#':
                    map_[move_first.r][move_first.c] = '.'
                if map_[move_second.r][move_second.c] == '#':
                    map_[move_second.r][move_second.c] = '.'
                # 누가 빨간색?
                red_one = (move_first if move_first.is_red else move_second)
                blue_one = (move_second if move_first.is_red else move_first)
                # 공들의 이동 결과에 대한 판단
                # 파란색 공이 들어가면 -> fail, 다음시도로 진행 -> continue
                if map_[blue_one.r][blue_one.c] == 'O':
                    continue
                # 파란색은 안들어가고 빨간색이 들어가면 -> success -> return
                elif map_[red_one.r][red_one.c] == 'O':
                    return turn
                # 둘다 안들어가면 일반적인 BFS 탐색 진행
                else:
                    if not visited[red_one.r][red_one.c][blue_one.r][blue_one.c]:
                        visited[red_one.r][red_one.c][blue_one.r][blue_one.c] = True
                        queue.append(BallPair(red_one, blue_one))
            size -= 1
        # turn 종료
        # 최대한 탐색은 10번까지만
        if turn == 10:
            break
        else:
            turn += 1
    return -1

R, C = map(int, input().split())

map_ = []
for r in range(R):
    map_.append(list(input()))
    for c in range(C):
        if map_[r][c] == 'R':
            red = Ball(r, c, True)
        elif map_[r][c] == 'B':
            blue = Ball(r, c, False)
# 입력 완료

deltas = [(-1, 0), (1, 0), (0, -1), (0, 1)] # 북, 남, 서, 동
        
print(bfs(red, blue))

7 7
#######
#...RB#
#.#####
#.....#
#####.#
#O....#
#######
5


파이썬의 문자열은 수정이 안되니까 주의하자

**수정이 되게끔 하기위해서 list(string)으로 사용하자**

visited = [False for _ in range(C)] 
visited = [visited for _ in range(R)]
visited = [visited for _ in range(C)]
visited = [visited for _ in range(R)]

위와 같은 방법으로 4차원 리스트를 만들면, [0,0,0,0]에만 값이 대입되는것이 아니라, 다른곳에도 함께 대입되어 문제가 생긴다.
이 방법은 리스트를 초기화 시킬때 모든 행이 같은 객체로 인식되기 때문이다.

visited = [[[[False] * C for _ in range(R)] for _ in range(C)] for _ in range(R)] # red r, red c, blue r, blue c

**위와 같이 리스트를 초기화 할 수 있도록 반드시 숙달하자.**

In [25]:
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

모든 변수를 초기화하는 문장도, 코딩테스트에서 필요하므로 암기하자